In [1]:
# Use R kernel
library(sumFREGAT)

In [ ]:
load("ref1KG.MAC5.EUR_AF.RData")

In [ ]:
# GWAS_OSR_cov_logistic.tsv is the result from 01_GWAS_OSR_cov_logistic.py, but need to add an additional column 
# of EA (Effect Allele, same as the alternative allele). 
# The header of GWAS_OSR_cov_logistic.tsv file is : CHROM	POS	ID	REF	ALT	P	Chi2	SE	BETA	EA
# This step will generate GWAS_OSR_cov_logistic.vcf.gz and GWAS_OSR_cov_logistic.vcf.gz.tbi.
# If it only generate GWAS_OSR_cov_logistic.vcf but not .gz and .tbi files, you can get them by running 
# the following two command yourself:
# bgzip -c file.vcf > file.vcf.gz
# tabix -p vcf file.vcf.gz

prep.score.files(
    data = './GWAS_OSR_cov_logistic.tsv',
    reference = "ref1KG.MAC5.EUR_AF.RData",
    output.file.prefix = "GWAS_OSR_cov_logistic"
)

In [ ]:
# SNP_cor_matrix/all/ folder contains SNP-SNP correlation matricies which can be downloaded 
# from https://mga.bionet.nsc.ru/sumFREGAT/
# refFlat_gene_SNPs.txt is a refFlat format file containing information of genomic positions and 
# spans of all genes in a genome, and can be downloaded from Dryad [https://doi.org/10.5061/dryad.vdncjsz43]

In [ ]:
# SKATO test
SKATO(score.file = 'GWAS_OSR_cov_logistic.vcf.gz', gene.file = "./refFlat_gene_SNPs.txt", cor.path = './SNP_cor_matrix/all',
     write.file = 'results/SKATO.txt', quiet = FALSE, beta.par=c(1,1))

In [ ]:
# PCA test
PCA(score.file = 'GWAS_OSR_cov_logistic.vcf.gz', gene.file = "./refFlat_gene_SNPs.txt", cor.path = './SNP_cor_matrix/all',
     write.file = 'results/PCA.txt', quiet = FALSE, n=431735)

In [ ]:
# ACAT-V test
ACAT(score.file = 'GWAS_OSR_cov_logistic.vcf.gz', gene.file = "./refFlat_gene_SNPs.txt",
     write.file = 'results/ACAT.txt', quiet = FALSE)

In [ ]:
# ACAT-O test
SKATO_df = read.table('./results/SKATO.txt',header = T)
PCA_df = read.table('./results/PCA.txt',header = T)
ACAT_df = read.table('./results/ACAT.txt',header = T)

p_combined_list <- list()

for (i in 1:nrow(ACAT_df)){
    p_SKATO = SKATO_df[i,'pvalue']
    p_PCA = PCA_df[i,'pvalue']
    p_ACAT = ACAT_df[i,'pvalue']
    p_combined <- ACATO(c(p_SKATO, p_PCA, p_ACAT))
    p_combined_list <- c(p_combined_list, p_combined)
}

ACATO_df <- ACAT_df

ACATO_df$pvalue <- as.numeric(p_combined_list)

write.table(ACATO_df, "results/ACATO.txt", row.names = FALSE,quote = FALSE)